Veiksmų eiga išskirta į dvi dalis - duomenų tvarkymą ir analizę.
<br>Galima "paleisti" visą notebook'ą (Run All).
<br>Kiekvienas žingsnis yra aprašytas - prielaidos, sprendimai ir pan.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

from plotly.subplots import make_subplots
from collections import Counter

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import time

from src.checks import print_limit_raw_values, check_missing_values
from src.helpers import fix_values_and_set_types, print_categorical_correlations

pd.set_option('display.max_columns', None)

## Duomenų tvarkymas <a class="anchor" id="first-bullet"></a>

In [2]:
raw_df = pd.read_csv("data/Telco_sample.csv",delimiter=';')
#nps_df = pd.read_csv("data/Telco_sample_NPS.csv",delimiter=';')

if len(set(raw_df['OFFER'])) == 1:
    print("Pašalinamas stulpelis 'OFFER', kadangi iš anksto žinoma, kad jis turi tik 1 galimą reikšmę.")
    raw_df.drop(columns=['OFFER'], inplace=True)

Pašalinamas stulpelis 'OFFER', kadangi iš anksto žinoma, kad jis turi tik 1 galimą reikšmę.


###

### Reikšmių patikrinimas, tipų nustatymas

In [3]:
check_only_nas = ['CUSTOMER_NR_H', 'SUBSCRIBER_NR_H', 'ADDRESS_ID_H', 'GRID_ID_500_H']

for col in check_only_nas:
    print(f"{col}\t number of NA values:{raw_df[col].isna().sum()}")

print_limit_raw_values(raw_df.drop(columns=check_only_nas))

CUSTOMER_NR_H	 number of NA values:0
SUBSCRIBER_NR_H	 number of NA values:0
ADDRESS_ID_H	 number of NA values:0
GRID_ID_500_H	 number of NA values:7983
PERIOD            	['2021.05.31', '2021.06.30'] - ['2022.03.31', '2022.04.30']
CUSTOMER_AGE_GROUP	['00. Unidentified', '01. < 25 y.o.'] - ['06. 65-74 y.o.', '07. 75+ y.o.']
CUSTOMER_GENDER   	['F', 'M', 'nan']
SUBSCRIBER_ON_DATE	['2019.01.02', '2019.01.03'] - ['2022.04.29', '2022.04.30']
SUBSCRIBER_OFF_DATE	['2021.05.03', '2021.05.04'] - ['2022.04.30', '3000.01.01']
SUBSCRIBER_STATUS 	['Active sim', 'Con/Disc'] - ['Recon/Disc', 'Reconnection']
PLAN_H            	['Plan 1', 'Plan 10'] - ['Plan 93', 'Plan 94']
PLAN_ON_DATE      	['.', '2019.01.02'] - ['2022.04.29', '2022.04.30']
PLAN_OFF_DATE     	['.', '2021.05.03'] - ['2022.05.27', '3000.01.01']
PLAN_CHANGE_FLAG  	['False', 'True']
OFFER_PREV        	['M2M', 'MBB'] - ['MHS', 'nan']
PLAN_PREV_H       	['Plan 1', 'Plan 10'] - ['Plan 97', 'nan']
TP_FLAG           	['False', 'True']
DATA_US

Reikšmės į kurias reikėtų atkreipti dėmesį - nan, '.', Unknown, neigiama FNF_NORM

Prielaida - "3000.01.01" reiškia kad sutartis (abonento ar plano) dar nėra nutraukta, data nežinoma.
<br>Prielaida - '.' žymimos nežinomos skaitinės ir datos reikšmės.
<br>Sutvarkomos '.' ir "3000.01.01" reikšmės - pakeičiama į NA.

In [4]:
typed_df = fix_values_and_set_types(raw_df)
#typed_df.info()

###

### Trūkstamų reikšmių analizė

In [5]:
display(check_missing_values(typed_df).sort_values('perc_missing', ascending=False))

,column,num_missing,perc_missing,num_unknown,perc_unknown
1,SUBSCRIBER_OFF_DATE,198323,0.9864,0,0.000
3,PLAN_OFF_DATE,194883,0.9692,0,0.000
10,SERVICE_CONTRACT_ON_DATE,73646,0.3663,0,0.000
11,SERVICE_CONTRACT_OFF_DATE,73646,0.3663,0,0.000
12,USED_DEVICE_TYPE,22612,0.1125,1215,0.006
13,USED_DEVICE_HIERARCHY_GROUP,22612,0.1125,1215,0.006
14,USED_DEVICE_HIERARCHY_TYPE,22612,0.1125,1215,0.006
15,GRID_ID_500_H,7983,0.0397,0,0.000
4,OFFER_PREV,5782,0.0288,0,0.000
5,PLAN_PREV_H,5782,0.0288,0,0.000


Kadangi ne visi klientai nutraukė abonentus, planus arba niekada neturėjo ar neturi kontrakto - normalu, kad šie stulpeliai turi daug tuščių reikšmių.
<br>Šių stulpelių trūkstamos reikšmės nėra tvarkomos, modeliuojant jas tikriausiai reikėtų įvertinti kaip nors kitaip.

<br> Prielaida - visas DEVICE ir GRID_ID_500_H trūkstamas reikšmes galima pakeisti į Unknown, nes jos tiesiog nėra žinomos.
<br> Prielaida - jeigu ankstesni planai ir kontraktai nežinomi - greičiausiai tai klientas jų tiesiog nėra turėjęs - None.
<br> Prielaida/klausimas - DEVICE stulpeliai jau turi galimą reikšmę "Unknown" - ar trūkstamos reikšmės reiškia, kad klientas nesinaudoja jokiu įrenginiu?

In [6]:

filled_df = typed_df.copy()
columns_set_nas_to_unknown = ['USED_DEVICE_TYPE','USED_DEVICE_HIERARCHY_GROUP','USED_DEVICE_HIERARCHY_TYPE', 'GRID_ID_500_H']
filled_df[columns_set_nas_to_unknown] = filled_df[columns_set_nas_to_unknown].fillna("Unknown")

filled_df['OFFER_PREV'] = filled_df['OFFER_PREV'].cat.add_categories('None')
filled_df['PLAN_PREV_H'] = filled_df['PLAN_PREV_H'].cat.add_categories('None')
filled_df[['OFFER_PREV','PLAN_PREV_H']] = filled_df[['OFFER_PREV','PLAN_PREV_H']].fillna("None")

# Nustatyti trūkstamą kliento lytį pagal paskutinę žinomą
filled_df['CUSTOMER_GENDER'] = filled_df.groupby('CUSTOMER_NR_H')['CUSTOMER_GENDER'].ffill()

# Jeigu duomenų suvartojimas nėra žinomas, jis yra lygus 0
filled_df['DATA_USAGE_MB'] = filled_df['DATA_USAGE_MB'].fillna(0)

# Nustatomas amžius pagal paskutinį žinomą
filled_df.loc[filled_df['CUSTOMER_AGE_GROUP'] == '00. Unidentified', 'CUSTOMER_AGE_GROUP'] = pd.NA
filled_df['CUSTOMER_AGE_GROUP'] = filled_df.groupby('CUSTOMER_NR_H')['CUSTOMER_AGE_GROUP'].ffill()

display(check_missing_values(filled_df).sort_values('perc_missing', ascending=False))

,column,num_missing,perc_missing,num_unknown,perc_unknown
2,SUBSCRIBER_OFF_DATE,198323,0.9864,0,0.0000
4,PLAN_OFF_DATE,194883,0.9692,0,0.0000
8,SERVICE_CONTRACT_ON_DATE,73646,0.3663,0,0.0000
9,SERVICE_CONTRACT_OFF_DATE,73646,0.3663,0,0.0000
5,MIN_BUCKET,310,0.0015,0,0.0000
6,SMS_BUCKET,310,0.0015,0,0.0000
7,DATA_BUCKET_MB,310,0.0015,0,0.0000
0,CUSTOMER_AGE_GROUP,203,0.0010,0,0.0000
1,CUSTOMER_GENDER,203,0.0010,0,0.0000
3,PLAN_ON_DATE,92,0.0005,0,0.0000


Prielaida - trūkstamų reikšmių liko palyginus mažai, būtų galima jas užpildyti artimiausių kaimynų metodu ar tiesiog konstanta. 
<br> Siekiant tikslumo ir nenorint išsiplėsti - pasirenkama pašalinti klientus (gal ne iki galo tikslinga), kurie turi šias likusias trūkstamas reikšmes.

In [7]:
# Gauti sąrašą klietų, kurie turi trūkstamas reikšmes
cols_with_understandable_nas = ['SERVICE_CONTRACT_ON_DATE', 'SERVICE_CONTRACT_OFF_DATE', 'PLAN_OFF_DATE', 'SUBSCRIBER_OFF_DATE']
missings_df = filled_df.drop(columns=cols_with_understandable_nas)
clients_with_nas = missings_df[missings_df.isna().any(1)]['CUSTOMER_NR_H']
# Pašalinti juos
filled_df = filled_df[~filled_df['CUSTOMER_NR_H'].isin(clients_with_nas)].reset_index(drop=True)

rows_before, rows_after = len(typed_df), len(filled_df)
users_before, users_after = len(set(typed_df['CUSTOMER_NR_H'])), len(set(filled_df['CUSTOMER_NR_H']))

print(f"Prarasta {rows_before-rows_after} iš {rows_before} ({round((rows_before-rows_after)/rows_before,4)*100}%) eilučių ir {users_before-users_after} iš {users_before} ({round((users_before-users_after)/users_before,4)*100}%) visų klientų.")

Prarasta 3010 iš 201067 (1.5%) eilučių ir 316 iš 16927 (1.87%) visų klientų.


###

### Tikrinama koreliacija 

Prielaida - atmetami kliento informacijos stulpeliai. Nors tikriausiai ADDRESS_ID_H gali koreliuoti su GRID_ID_500_H...?

In [8]:
cols_to_skip = ['PERIOD', 'CUSTOMER_NR_H', 'CUSTOMER_AGE_GROUP', 'CUSTOMER_GENDER', 'SUBSCRIBER_NR_H', 'ADDRESS_ID_H', 'GRID_ID_500_H']
test_corr_df = filled_df.copy()
test_corr_df = test_corr_df.drop(columns=cols_to_skip)
test_corr_df.drop_duplicates(inplace=True)
test_corr_df.corr()

,PLAN_CHANGE_FLAG,TP_FLAG,DATA_USAGE_MB,MIN_BUCKET,SMS_BUCKET,DATA_BUCKET_MB,USED_DEVICE_FLAG,FMF_NORM
PLAN_CHANGE_FLAG,1.000000,0.031710,0.006643,0.041657,0.044096,0.011613,0.013179,0.105158
TP_FLAG,0.031710,1.000000,0.061765,0.027772,0.027808,-0.002004,0.013442,0.037711
DATA_USAGE_MB,0.006643,0.061765,1.000000,0.139541,0.116120,0.085756,0.044023,0.340735
MIN_BUCKET,0.041657,0.027772,0.139541,1.000000,0.826745,0.018155,0.315315,0.456819
SMS_BUCKET,0.044096,0.027808,0.116120,0.826745,1.000000,0.016730,0.361764,0.410616
DATA_BUCKET_MB,0.011613,-0.002004,0.085756,0.018155,0.016730,1.000000,0.007385,0.122644
USED_DEVICE_FLAG,0.013179,0.013442,0.044023,0.315315,0.361764,0.007385,1.000000,0.162510
FMF_NORM,0.105158,0.037711,0.340735,0.456819,0.410616,0.122644,0.162510,1.000000


Galbūt SMS_BUCKET kažkiek koreliuoja su MIN_BUCKET, bet kol kas paliekama.

In [9]:
print_categorical_correlations(test_corr_df, 0.8)

Cramer's V:
PLAN_H - PLAN_PREV_H                  0.8019536850878781
OFFER_PREV - PLAN_PREV_H                  1.0
PLAN_PREV_H - PLAN_H                       0.8019536850878781
PLAN_PREV_H - OFFER_PREV                   1.0
USED_DEVICE_TYPE - USED_DEVICE_HIERARCHY_GROUP  0.9254920344970172
USED_DEVICE_TYPE - USED_DEVICE_HIERARCHY_TYPE   0.9997535834453323
USED_DEVICE_HIERARCHY_GROUP - USED_DEVICE_TYPE             0.9254920344970172
USED_DEVICE_HIERARCHY_GROUP - USED_DEVICE_HIERARCHY_TYPE   1.0
USED_DEVICE_HIERARCHY_TYPE - USED_DEVICE_TYPE             0.9997535834453324
USED_DEVICE_HIERARCHY_TYPE - USED_DEVICE_HIERARCHY_GROUP  1.0


Didelė koreliacija tarp DEVICE kintamųjų ir PLAN/OFFER/PREV kintamųjų - galbūt daugumoj atvejų planas tiesiog nesikeitė?

In [10]:
high_correlation_columns = ['PLAN_H', 'PLAN_PREV_H', 'OFFER_PREV', 'USED_DEVICE_HIERARCHY_TYPE', 'USED_DEVICE_TYPE', 'USED_DEVICE_HIERARCHY_GROUP']
test_corr_df[high_correlation_columns].describe()

,PLAN_H,PLAN_PREV_H,OFFER_PREV,USED_DEVICE_HIERARCHY_TYPE,USED_DEVICE_TYPE,USED_DEVICE_HIERARCHY_GROUP
count,143796,143796,143796,143796,143796,143796
unique,42,92,4,20,13,8
top,Plan 1,Plan 1,MHS,High-end smartphone,Mobile phone,Smartphone
freq,47497,45531,139148,107986,135828,132125


Dėl kategorijų gausos nusprendžiama palikti USED_DEVICE_HIERARCHY_TYPE, kadangi kitais atvejais ~95% įrašų dominuoja 1 kategorija.

In [11]:
display(filled_df['PLAN_H'].value_counts().head(10))
print(f"Planas keitėsi {round(filled_df['PLAN_CHANGE_FLAG'].value_counts()[1]/len(filled_df['PLAN_CHANGE_FLAG']),4)*100}% atvejų")

Plan 1     49223
Plan 2     38839
Plan 3     28513
Plan 4     20575
Plan 5     12714
Plan 6      8895
Plan 7      7231
Plan 9      6271
Plan 11     4062
Plan 10     3710
Name: PLAN_H, dtype: int64

Planas keitėsi 2.45% atvejų


Žinant daugiau apie planus (jų charakteristikas), būtų galima apjungti juos į grupes ir sukurti vieną kintamąjį vietoj trijų: PLAN_GROUPS_CHANGED. Dabar dėl reto planų keitimo, paliekamas tik plano keitimo faktas.
<br> SERVICE_CONTRACT_STATUS gerai atspindi datas, kada kontraktas pradėtas ir pabaigtas, jos pašalinamos. (datos taip pat turi daug trūkstamų reikšmių)

In [16]:
print(f"{round(filled_df['SERVICE_CONTRACT_ON_DATE'].isna().sum()/len(filled_df),4)*100}% įrašų neturi kontrakto pradžios datos.")

36.69% įrašų neturi kontrakto pradžios datos.


In [17]:
to_drop = ['USED_DEVICE_TYPE', 'USED_DEVICE_HIERARCHY_GROUP', 'OFFER_PREV', 'PLAN_H', 'PLAN_PREV_H', 'SERVICE_CONTRACT_ON_DATE', 'SERVICE_CONTRACT_OFF_DATE']
no_corr_df = filled_df.copy()
no_corr_df.drop(columns=to_drop, inplace=True)

TODO: patikrinti ar koreliuoja kitos datos. Galbūt planų ir abonentų datos yra stipriai susijusios ir kažkurių būtų galima atsisakyti arba kaip nors apjungti.

###

### Išskirčių tikrinimas

In [18]:
no_corr_df.describe()

,DATA_USAGE_MB,MIN_BUCKET,SMS_BUCKET,DATA_BUCKET_MB,FMF_NORM
count,198057.000000,198057.000000,198057.000000,1.980570e+05,198057.000000
mean,6287.017649,732873.507889,782109.647415,1.190251e+05,0.241191
std,16593.115714,442442.857876,412809.860403,2.119987e+06,0.177913
min,0.000000,0.000000,0.000000,0.000000e+00,-0.080270
25%,0.000000,200.000000,999999.000000,2.000000e+02,0.055180
50%,1024.000000,999999.000000,999999.000000,8.192000e+03,0.248490
75%,6850.560000,999999.000000,999999.000000,2.048000e+04,0.331440
max,659667.100000,999999.000000,999999.000000,1.048781e+08,0.936790


In [19]:
print(f"{len(no_corr_df[no_corr_df['FMF_NORM'] < 0])} įrašų iš {len(no_corr_df)} ({round(len(no_corr_df[no_corr_df['FMF_NORM'] < 0])/len(no_corr_df),4)*100}%) FMF_NORM reikšmė yra neigiama")
no_corr_df['FMF_NORM'] = no_corr_df['FMF_NORM'].clip(lower=0)

3007 įrašų iš 198057 (1.52%) FMF_NORM reikšmė yra neigiama


Prielaida - nežinau ar gali taip būti, galbūt normalizuotos reikšmės būtų nuo 0 iki 1 (?). Nustatoma į 0.

TODO: Atlikti kitų kintamųjų išskirčių tikrinimą (nors iki šiol dalinai jau buvo pražiūrėta)
<br> Galima išsibraižyti box plots arba atsifiltruoti reikšmes pagal IQR metodą.

###

In [16]:
#no_corr_df.to_pickle(f'{time.now()}_cleaned_data.pkl')

# Analitika <a class="anchor" id="second-bullet"></a>

In [21]:
#df = pd.read_pickle('...._cleaned_data.pkl')
df = no_corr_df.copy()
plot_gender_color_map = {'M': 'cornflowerblue', 'F': 'darksalmon'}

#### Amžiaus grupė ir lytis

In [35]:
users = df[['CUSTOMER_NR_H', 'CUSTOMER_AGE_GROUP', 'CUSTOMER_GENDER', 'ADDRESS_ID_H', 'GRID_ID_500_H']].drop_duplicates()
num_unique_users = len(set(users['CUSTOMER_NR_H']))
print(f"Per tyrimo laikotarpį turimi duomenys (bent 1 mėn.) apie {num_unique_users} unikalių klientų.")

users_changed = len(users) - len(set(users['CUSTOMER_NR_H']))
print(f"Apie {users_changed} klientų ({round(users_changed/num_unique_users,5)*100}% visų klientų) per tyrimo laikotarpį pakeitė amžiaus grupę / gyv. vietą / lytį?")

unique_users_df = users.groupby('CUSTOMER_NR_H').last().reset_index()

print(dict(Counter(unique_users_df['CUSTOMER_GENDER'])))

age_gender_df = unique_users_df.groupby(by=['CUSTOMER_GENDER', 'CUSTOMER_AGE_GROUP']).agg(
    counts = pd.NamedAgg(column='CUSTOMER_NR_H', aggfunc='count')
).reset_index(drop=False)

fig = px.histogram(age_gender_df, x="CUSTOMER_AGE_GROUP", y="counts",
             color='CUSTOMER_GENDER', barmode='group',
             height=350,
             width=1000,
             text_auto=True,
             color_discrete_map=plot_gender_color_map
             ).update_layout(
                yaxis_title="Number of customers",
                xaxis_title="Customer age group",
                legend_title="Customer gender"
             )
fig.show()

Per tyrimo laikotarpį turimi duomenys (bent 1 mėn.) apie 16611 unikalių klientų.
Apie 1220 klientų (7.345% visų klientų) per tyrimo laikotarpį pakeitė amžiaus grupę / gyv. vietą / lytį?
{'M': 8234, 'F': 8377}


### Klientų ilgaamžiškumas (lifetime)

Prielaida/terminologija - "išeinančiais" klientais vadinami tie, kurių paskutinis įrašas buvo iki tyrimo periodo galo, t.y. neturėjo įrašo '2022-04-30'.

In [36]:
# Nustatoma kiekvieno kliento pirmo ir paskutinio įrašų datos bei kelių mėnesių duomenys yra žinomi
users_lifetimes = df[['PERIOD', 'CUSTOMER_NR_H']].drop_duplicates().groupby('CUSTOMER_NR_H').agg(
    min_date = pd.NamedAgg(column='PERIOD', aggfunc='min'),
    max_date = pd.NamedAgg(column='PERIOD', aggfunc='max'),
    count = pd.NamedAgg(column='PERIOD', aggfunc='count')
).reset_index(drop=False)

# Ar tyrimo gale klientas pasiliko (yra įrašas iš tyrimo laikotarpio galo - 2022-04-30)
users_lifetimes["Stayed"] = False
users_lifetimes.loc[users_lifetimes['max_date'] == '2022-04-30', "Stayed"] = True

qty_less_1yr = len(users_lifetimes[users_lifetimes['count'] < 12])
print(f"Duomenys apie {qty_less_1yr} klientus ({round(qty_less_1yr/len(users_lifetimes),5)*100}%) yra iš trumpesnio periodo nei metai - nauji arba išeinantys klientai.")  

mask_quit = ~users_lifetimes['Stayed']
mask_stayed = users_lifetimes['Stayed']
mask_joined_later = users_lifetimes['min_date'] != '2021-05-31'
mask_old_users = users_lifetimes['min_date'] == '2021-05-31'

old_users_quit = users_lifetimes[(mask_old_users) & (mask_quit)]
print(f"{len(old_users_quit)} ({round(len(old_users_quit)/len(users_lifetimes),4)*100}%) buvę klientai išėjo per tyrimo laikotarpį.")

users_joined_and_quit = users_lifetimes[(mask_joined_later) & (mask_quit)]
print(f"{len(users_joined_and_quit)} ({round(len(users_joined_and_quit)/len(users_lifetimes),4)*100}%) klientai(-ų) prisijungė ir išėjo dar tais pačiais metais (per tyrimo laikotarpį). Jų vid. buvimo laikas: {round(users_joined_and_quit['count'].mean(),2)} mėn.")

users_joined_and_stayed = users_lifetimes[(mask_joined_later) & (mask_stayed)]
print(f"{len(users_joined_and_stayed)} ({round(len(users_joined_and_stayed)/len(users_lifetimes),4)*100}%) klientai prisijungę per tyrimo laikotarpį ir liko bent iki tyrimo galo.")

users_with_gaps = users_lifetimes[(mask_old_users) & (mask_stayed) & (users_lifetimes['count'] < 12)]
print(f"{len(users_with_gaps)} ({round(len(users_with_gaps)/len(users_lifetimes),4)*100}%) klientų duomenyse yra tarpų (trūksta mėnesių).")

fig = px.histogram(users_joined_and_quit['count'],
              histnorm='percent',
              width=700,
              height=400,
              text_auto='perc_plot',
              title='Users who joined and left during the period'
              ).update_layout(
                bargap=0.3,
                yaxis_title="Number (percent) of new users",
                xaxis_title="Number of months after which new user quit",
                showlegend=False
             )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False, texttemplate='%{y:.2f}%')

fig.show()

Duomenys apie 6109 klientus (36.777%) yra iš trumpesnio periodo nei metai - nauji arba išeinantys klientai.
1542 (9.28%) buvę klientai išėjo per tyrimo laikotarpį.
315 (1.9%) klientai(-ų) prisijungė ir išėjo dar tais pačiais metais (per tyrimo laikotarpį). Jų vid. buvimo laikas: 3.34 mėn.
4179 (25.16%) klientai prisijungę per tyrimo laikotarpį ir liko bent iki tyrimo galo.
73 (0.44%) klientų duomenyse yra tarpų (trūksta mėnesių).


Nauji išeinantys klientai dažniausiai (32.06% atvejų) išeina po 2 mėn. nuo prisijungimo.

Tikslas - suagreguoti klientų duomenis laike į apibendrinančius kintamuosius.
<br> * Atrenkami klientai, kurie tyrimo pradžioje jau buvo, bet po 4 ar daugiau mėnesių išėjo - išeinantys klientai;
<br> * Atrenkama tiek pat "gerųjų" klientų, apie kuriuos turimi duomenys nuo pradžios iki galo.
<br> * Galutinis duomenų rinkinys agreguojamas kliento lygiu

In [50]:
# Atrenkamas sąrašas senų klientų, kurie tyrimo laikotarpiu išbuvo 4 mėnesius ir ilgiau, bet iki tyrimo galo išėjo
old_users_quit_wip = list(old_users_quit[old_users_quit['count'] >= 4]['CUSTOMER_NR_H'])

# Iš pradinės duomenų imties atrenkami duomenys laike apie likusius ir išėjusius klientus
# Imama tiek pat likusių klientų, kiek surasta išėjusių
old_stayed_users_sample = list(users_lifetimes[(mask_old_users) & (mask_stayed) & (users_lifetimes['count'] == 12)].sample(len(old_users_quit_wip))['CUSTOMER_NR_H'])
sample_with_time_df = df[df['CUSTOMER_NR_H'].isin(old_users_quit_wip+old_stayed_users_sample)]
sample_with_time_df = sample_with_time_df.sort_values(['CUSTOMER_NR_H', 'PERIOD']).reset_index(drop=True)

# Sukuriamas galutinis duomenų rinkinys kuriame bus agreguojami klientų duomenys
agg_user_smaple_df = unique_users_df[unique_users_df['CUSTOMER_NR_H'].isin(old_users_quit_wip+old_stayed_users_sample)][['CUSTOMER_NR_H','CUSTOMER_AGE_GROUP','CUSTOMER_GENDER']].reset_index(drop=True)
agg_user_smaple_df['stayed'] = agg_user_smaple_df['CUSTOMER_NR_H'].isin(old_stayed_users_sample)
agg_user_smaple_df = agg_user_smaple_df.merge(users_lifetimes[['CUSTOMER_NR_H', 'max_date', 'count']], on='CUSTOMER_NR_H', how='left')
agg_user_smaple_df.rename(inplace=True, columns={"count":"num_unique_months", 'max_date':'last_record'})

print(f"Toliau nagrinėjami {len(agg_user_smaple_df)} klientai")

Toliau nagrinėjami 2138 klientai


Agreguojami dydžiai:
<br>* Kliento abonentų skaičius;
<br>* Kiek mėnesių turėjo kontraktą (normuota pagal visų įrašų skaičių);
<br>* Kiek kartų keitė planą (normuota pagal visų įrašų skaičių);
<br>* Vidutinis duomenų suvartojimas per visą laiką;
<br>* ....

In [51]:
agg_user_smaple_df['total_records'] = agg_user_smaple_df['CUSTOMER_NR_H'].map(dict(Counter(df['CUSTOMER_NR_H'])))
sample_with_time_df['had_contract'] = True
sample_with_time_df.loc[sample_with_time_df['SERVICE_CONTRACT_STATUS'] == 'No contract', 'had_contract'] = False

subs_counts = sample_with_time_df.groupby(['CUSTOMER_NR_H', 'PERIOD']).size().reset_index(name='num_of_subs')
subs_counts_agg = subs_counts.groupby('CUSTOMER_NR_H').agg(
    median_subs = pd.NamedAgg(column='num_of_subs', aggfunc='median'),
    avg_subs = pd.NamedAgg(column='num_of_subs', aggfunc='mean'),
    max_subs = pd.NamedAgg(column='num_of_subs', aggfunc='max'),
).reset_index()


plan_change_counts = sample_with_time_df.groupby('CUSTOMER_NR_H').agg(
        num_plan_changes=pd.NamedAgg(column='PLAN_CHANGE_FLAG', aggfunc='sum'),
        avg_all_time_data_usage=pd.NamedAgg(column='DATA_USAGE_MB', aggfunc='mean'),
        num_months_with_contract=pd.NamedAgg(column='had_contract', aggfunc='sum'),
        avg_mb_bucket=pd.NamedAgg(column='DATA_BUCKET_MB', aggfunc='mean'),
        avg_fmf = pd.NamedAgg(column='FMF_NORM', aggfunc='mean'),
        median_fmf = pd.NamedAgg(column='FMF_NORM', aggfunc='median'),
    ).reset_index(drop=False)

agg_user_smaple_df = agg_user_smaple_df.merge(plan_change_counts, on='CUSTOMER_NR_H').merge(subs_counts_agg, on='CUSTOMER_NR_H', how='left')

agg_user_smaple_df['plan_change_eval'] = agg_user_smaple_df['num_plan_changes']/agg_user_smaple_df['total_records']
agg_user_smaple_df['contract_eval'] = agg_user_smaple_df['num_months_with_contract']/agg_user_smaple_df['total_records']

<br>* Duomenų suvartojimo pokytis paskutiniais mėnesiais (2mėn_vidurkis - t-2_2mėn_vidurkis)

In [53]:
N_MONTHS = 2 # Kelių paskutinių mėnesių vidurkius lyginti

def difference_in_averages(df, n_months):
    df = df.sort_values(by='PERIOD')
    last_date = df['PERIOD'].max()

    back_N_months = last_date - pd.DateOffset(months=n_months) - pd.DateOffset(days=4) + pd.offsets.MonthEnd(0)
    avg_last_N_months = df[df['PERIOD'] > back_N_months]['DATA_USAGE_MB'].mean()

    back_2N_months = last_date - pd.DateOffset(months=n_months*2) - pd.DateOffset(days=4) + pd.offsets.MonthEnd(0)

    df_previous = df[(back_2N_months < df['PERIOD']) & (df['PERIOD'] <= back_N_months)]
    avg_previous_N_months = df_previous['DATA_USAGE_MB'].mean()
    
    if (len(df[df['PERIOD'] > back_N_months]) < N_MONTHS) or (len(df_previous) < N_MONTHS):
        print("Less than 2 months of history:", len(df[df['PERIOD'] > back_N_months]), (len(df_previous)))
        avg_previous_N_months = avg_last_N_months # Dirty fix

    delta_N_months = avg_last_N_months - avg_previous_N_months
    return delta_N_months


diff_in_averages = sample_with_time_df.groupby('CUSTOMER_NR_H').apply(difference_in_averages, n_months=N_MONTHS).reset_index(name=f'diff_in_{N_MONTHS}month_avgs')
agg_user_smaple_df = agg_user_smaple_df.merge(diff_in_averages, on='CUSTOMER_NR_H', how='left')

Less than 2 months of history: 2 1
Less than 2 months of history: 2 1
Less than 2 months of history: 2 0


Apibendrinimo vizualizacijos

In [54]:
stayed_df = agg_user_smaple_df[agg_user_smaple_df['stayed']]
quit_df = agg_user_smaple_df[~agg_user_smaple_df['stayed']]
print(f"Vidutinis duomenų suvartojimo skirtumas. Paskutinių {N_MONTHS} mėnesių vidurkis lyginant su t-{N_MONTHS} {N_MONTHS} mėnesių vidurkiu.\n Likę klientai: {stayed_df[f'diff_in_{N_MONTHS}month_avgs'].mean()} \n Išėję klientai: {quit_df[f'diff_in_{N_MONTHS}month_avgs'].mean()}")
print(f"\nVidutinė FMF_NORM reikšmė.\n Likę klientai: {stayed_df['avg_fmf'].mean()} \n Išėję klientai: {quit_df[f'avg_fmf'].mean()}")

df_cut_to_plot = agg_user_smaple_df[(agg_user_smaple_df[f'diff_in_{N_MONTHS}month_avgs'].between(-80000,80000))]
print(f"\nVaizduojami {len(df_cut_to_plot)} klientai iš {len(agg_user_smaple_df)}")

fig = px.scatter(
        df_cut_to_plot, 
        x=f'diff_in_{N_MONTHS}month_avgs',
        y='avg_fmf', 
        color ='stayed',
        color_discrete_sequence={True:'darksalmon', False:'cornflowerblue'},
        title=f'Users ({len(stayed_df)} old and quit, {len(quit_df)} old and stayed) average internet usage change vs average FMF',
        width=1100
    ).update_layout(
        yaxis_title="Average user FMF_NORM",
        xaxis_title=f"User internet usage change ({N_MONTHS} last months avg. - {N_MONTHS} months before avg.)",
    ).update_traces(marker={'size': 4.5}
)

fig.show()

Vidutinis duomenų suvartojimo skirtumas. Paskutinių 2 mėnesių vidurkis lyginant su t-2 2 mėnesių vidurkiu.
 Likę klientai: 854.4693858635129 
 Išėję klientai: -599.1321701746181

Vidutinė FMF_NORM reikšmė.
 Likę klientai: 0.264662801608306 
 Išėję klientai: 0.19538530501405302

Vaizduojami 2132 klientai iš 2138


In [55]:
mask_stayed = agg_user_smaple_df['stayed']

fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Histogram(x=agg_user_smaple_df[mask_stayed]['contract_eval'], name='stayed', marker_color='darksalmon',histnorm='percent'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=agg_user_smaple_df[~mask_stayed]['contract_eval'], name='quit', marker_color='cornflowerblue',histnorm='percent'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=agg_user_smaple_df[~mask_stayed]['plan_change_eval'], marker_color='cornflowerblue',histnorm='percent',showlegend=False,
    xbins=dict(
        start=0,
        end=1,
        size=0.02
    )),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=agg_user_smaple_df[mask_stayed]['plan_change_eval'], marker_color='darksalmon',histnorm='percent',showlegend=False,
    xbins=dict(
        start=0,
        end=1,
        size=0.02
    )),
    row=1, col=2
)
fig.update_layout(title='Users distribution based on contracts and plan changes')
fig.update_yaxes(title_text='% of users',row=1, col=1)
fig.update_xaxes(title_text='Average contracts per month per subscriber',row=1, col=1)
fig.update_yaxes(title_text='% of users',row=1, col=2)
fig.update_xaxes(title_text='Average plan changes per month per subscriber',row=1, col=2)
fig.show()

###